In [6]:
import json
import firebase_admin
from firebase_admin import firestore
import os


def fetch_collection(collection_name):
    os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = "/Users/thanhlam/Documents/Projects/nomnom/scraper/creds/nomnom-a494d-firebase-adminsdk-psvbg-dfcd9bc651.json"

    db = firestore.Client()

    docs = db.collection(collection_name).stream()

    results = {}
    for doc in docs:
        results[doc.id] = doc.to_dict()

    return results

# Name of your Firestore collection
collection_name = "eateries"

# Fetch all documents from the collection
collection_data = fetch_collection(collection_name)

# Store the data in a JSON file
with open(f"{collection_name}.json", "w") as file:
    json.dump(collection_data, file, ensure_ascii=False)

In [14]:
import json
import csv

with open("eateries.json", "r") as file:
    eateries = json.load(file)
    for k,v in eateries.items():
        if(not v["posts"]):
            print(k)
        

manifestomarket.prague
mezisrnky_vinohrady
muj.bagel
sansho.cz


In [ ]:
import itertools
import shutil
from time import sleep
import requests
import json


def download_image(media_url, image_path):
    response = requests.get(media_url, stream=True)
    if response.status_code == 200:
        with open(image_path, 'wb') as file:
            response.raw.decode_content = True
            shutil.copyfileobj(response.raw, file)

with open("dataset_instagram-post-scraper_2023-06-16_19-15-22-631.json", "r") as posts_file:
    with open("eateries.json", "r+") as eateries_file:
        eateries = json.load(eateries_file)
        posts = json.load(posts_file)
        grouped_posts = itertools.groupby(sorted(posts, key=lambda p:p["ownerUsername"]), lambda p:p["ownerUsername"])
        grouped_posts_dict = {key: list(group) for key, group in grouped_posts}
        # with open("grouped_posts.json", "w") as file:
        #     json.dump(grouped_posts_dict, file)
        for k, v in eateries.items():
            if v["posts"]:
                continue
            ig_username = v["ig_username"]
            try:
                acc_posts = grouped_posts_dict[ig_username]
            except KeyError:
                continue
            v["posts"] =list(map(lambda p: {
                "caption": p["caption"],
                "post_url": p["url"],
                "image_url": p["displayUrl"],
                "price": None
            } ,acc_posts))
            for post in v["posts"]:
                try:
                    shortCode = post["post_url"].split("/")[-2]
                    print(f"processing {ig_username} - {post['post_url']}")
                    download_image(post["image_url"], f"./images/{ig_username}-{shortCode}.jpg")
                    post["image_url"] = f"{ig_username}-{shortCode}.jpg"
                except Exception as e:
                    print(f"couldn't download image: {e}")
                    
        json.dump(eateries, eateries_file, ensure_ascii=False)




In [6]:
import json
import requests

def get_place_details(place_id, api_key):
    url = "https://maps.googleapis.com/maps/api/place/details/json"
    params = {
        "place_id": place_id,
        "fields": "name,rating,formatted_phone_number,opening_hours,price_level,website",
        "key": api_key
    }
    response = requests.get(url, params=params)
    while response.status_code != 200:
        print("response failed - trying again")
        sleep(10)
        response = requests.get(url, params=params)
    try:
        return response.json()['result']
    except KeyError:
        print(response.json())

# Your Google Places API key
api_key = "<api-key>"

# Read the restaurant objects from a JSON file
with open('./data/eateries.json', 'r') as file:
    restaurants = json.load(file)

# Fetch additional information for each restaurant
for _,restaurant in restaurants.items():
    print(f"processing {restaurant['name']}")
    place_details = get_place_details(restaurant['location']['google_place_id'], api_key)
    if not place_details:
        restaurant["delete"] = True
        continue
    restaurant["contact"] = {
        "phone_number": place_details.get("formatted_phone_number", None),
        "website": place_details.get("website", None),
        "opening_hours": place_details.get("opening_hours", None),
        "ig": restaurant["ig_username"]
    }
    del restaurant["ig_username"]
    restaurant["rating"] = place_details.get("rating", None)
    restaurant["price_level"] = place_details.get("price_level", None)

# Write the extended restaurant objects back to the JSON file
with open('./data/restaurants.json', 'r+') as file:
    json.dump(restaurants, file, ensure_ascii=False)

In [10]:
import json
from firebase_admin import firestore
import os


def add_to_collection(collection_name, data):
    os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = "/Users/thanhlam/Documents/Projects/nomnom/scraper/creds/nomnom-a494d-firebase-adminsdk-psvbg-dfcd9bc651.json"
    db = firestore.Client()
    for k,v in data.items():
        db.collection(collection_name).document(k).set(v)

with open("./data/restaurants.json", "r") as file:
    restaurants = json.load(file)
    for k,v in restaurants.items():
        for p in v["posts"]:
            p["image_url"] = "images/" + p["image_url"]



collection_name = "eateries"

add_to_collection(collection_name, restaurants)